In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00


In [4]:
 import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
import optuna
from PIL import Image
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset
import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
auswahl_model = "FSRCNN" #oder SRCNN

In [10]:
low_res_folder = "./drive/MyDrive/Upscaling/mensch_low_res"
high_res_folder = "./drive/MyDrive/Upscaling/mensch"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



from tqdm import tqdm

# === Normalization ===
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

train_dataset = [(normalize(image), normalize(target)) for image, target in tqdm(train_dataset, desc="Normalizing Train Dataset")]

val_dataset = [(normalize(image), normalize(target)) for image, target in tqdm(val_dataset, desc="Normalizing Validation Dataset")]





Normalizing Validation Dataset: 100%|██████████| 230/230 [02:11<00:00,  1.75it/s]

Number of training samples originally: 918
Number of val samples: 230


In [28]:
# === final data loaders ===

# Data loaders for train and val sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples originally: {len(train_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")


Number of training samples originally: 918
Number of val samples: 230


In [11]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=3, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

In [12]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=5, padding=4, output_padding=4)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [ ]:
import optuna
num_epochs = 20

def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if auswahl_model == "FSRCNN":
      model = FSRCNN().to(device)
    else:
      model = SRCNN().to(device)

    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    d = trial.suggest_int('d', 32, 128)
    s = trial.suggest_int('s', 5, 20)
    m = trial.suggest_int('m', 2, 8)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=20)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()

# Speichern des DataFrames als CSV-Datei
if auswahl_model == "FSRCNN":
  df.to_csv('study_results_FSRCNN.csv', index=False)
else:
  df.to_csv('study_results_SRCNN.csv', index=False)

[I 2023-06-22 15:44:22,476] A new study created in memory with name: no-name-16bf4101-c7e9-4aa4-98fd-787207315db6
<ipython-input-29-56b35adcda66>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)


Epoch [1/20], Training Loss: 0.2009, Validation Loss: 0.0885, PSNR: 10.528818761984883
Epoch [2/20], Training Loss: 0.0807, Validation Loss: 0.0702, PSNR: 11.537680049783667
Epoch [3/20], Training Loss: 0.0637, Validation Loss: 0.0552, PSNR: 12.581413594394181
Epoch [4/20], Training Loss: 0.0532, Validation Loss: 0.0485, PSNR: 13.146560981029406
Epoch [5/20], Training Loss: 0.0481, Validation Loss: 0.0442, PSNR: 13.546757897742408
Epoch [6/20], Training Loss: 0.0435, Validation Loss: 0.0395, PSNR: 14.0347425585427
Epoch [7/20], Training Loss: 0.0345, Validation Loss: 0.0290, PSNR: 15.368977929365768
Epoch [8/20], Training Loss: 0.0278, Validation Loss: 0.0255, PSNR: 15.930452632390713
Epoch [9/20], Training Loss: 0.0254, Validation Loss: 0.0236, PSNR: 16.2779761781966
Epoch [10/20], Training Loss: 0.0238, Validation Loss: 0.0224, PSNR: 16.491546798424597
Epoch [11/20], Training Loss: 0.0228, Validation Loss: 0.0215, PSNR: 16.682109093394747
Epoch [12/20], Training Loss: 0.0219, Validat

[I 2023-06-22 15:44:49,267] Trial 0 finished with value: 0.017801506677642465 and parameters: {'lr': 0.0014888843294295106, 'd': 101, 's': 15, 'm': 8}. Best is trial 0 with value: 0.017801506677642465.


Epoch [20/20], Training Loss: 0.0186, Validation Loss: 0.0178, PSNR: 17.49543238472321
Epoch [1/20], Training Loss: 0.1828, Validation Loss: 0.0869, PSNR: 10.612040303742777
Epoch [2/20], Training Loss: 0.0777, Validation Loss: 0.0667, PSNR: 11.759723208254291
Epoch [3/20], Training Loss: 0.0595, Validation Loss: 0.0517, PSNR: 12.861140935740107
Epoch [4/20], Training Loss: 0.0502, Validation Loss: 0.0448, PSNR: 13.484732615491971
Epoch [5/20], Training Loss: 0.0375, Validation Loss: 0.0298, PSNR: 15.264091349991668
Epoch [6/20], Training Loss: 0.0283, Validation Loss: 0.0252, PSNR: 15.989101917793763
Epoch [7/20], Training Loss: 0.0251, Validation Loss: 0.0233, PSNR: 16.32522460408436
Epoch [8/20], Training Loss: 0.0234, Validation Loss: 0.0218, PSNR: 16.618067922927597
Epoch [9/20], Training Loss: 0.0220, Validation Loss: 0.0207, PSNR: 16.835349270193696
Epoch [10/20], Training Loss: 0.0211, Validation Loss: 0.0199, PSNR: 17.01132321263495
Epoch [11/20], Training Loss: 0.0206, Valida

[I 2023-06-22 15:45:16,232] Trial 1 finished with value: 0.017252078978344798 and parameters: {'lr': 0.0023175389221966733, 'd': 110, 's': 10, 'm': 4}. Best is trial 1 with value: 0.017252078978344798.


...

Epoch [20/20], Training Loss: 0.0188, Validation Loss: 0.0180, PSNR: 17.439312308768553
Epoch [1/20], Training Loss: 0.1769, Validation Loss: 0.0784, PSNR: 11.054358071015391
Epoch [2/20], Training Loss: 0.0671, Validation Loss: 0.0542, PSNR: 12.662310256169064
Epoch [3/20], Training Loss: 0.0506, Validation Loss: 0.0448, PSNR: 13.488660810919269
Epoch [4/20], Training Loss: 0.0436, Validation Loss: 0.0392, PSNR: 14.069098373072151
Epoch [5/20], Training Loss: 0.0376, Validation Loss: 0.0381, PSNR: 14.191203037146966
Epoch [6/20], Training Loss: 0.0281, Validation Loss: 0.0236, PSNR: 16.274215551194075
Epoch [7/20], Training Loss: 0.0233, Validation Loss: 0.0219, PSNR: 16.603265439493754
Epoch [8/20], Training Loss: 0.0220, Validation Loss: 0.0208, PSNR: 16.812115261539073
Epoch [9/20], Training Loss: 0.0212, Validation Loss: 0.0205, PSNR: 16.880880826135428
Epoch [10/20], Training Loss: 0.0330, Validation Loss: 0.0367, PSNR: 14.348965967643199
Epoch [11/20], Training Loss: 0.0264, Val

[I 2023-06-22 15:48:55,319] Trial 9 finished with value: 0.016618059016764164 and parameters: {'lr': 0.0036112867615120015, 'd': 116, 's': 12, 'm': 6}. Best is trial 6 with value: 0.01646598894149065.


Epoch [20/20], Training Loss: 0.0171, Validation Loss: 0.0166, PSNR: 17.79419703026967
Epoch [1/20], Training Loss: 0.3398, Validation Loss: 0.1229, PSNR: 9.103569162012782
Epoch [2/20], Training Loss: 0.0896, Validation Loss: 0.0706, PSNR: 11.513073486699675
Epoch [3/20], Training Loss: 0.0636, Validation Loss: 0.0542, PSNR: 12.662615384284404
Epoch [4/20], Training Loss: 0.0518, Validation Loss: 0.0455, PSNR: 13.419356167181377
Epoch [5/20], Training Loss: 0.0415, Validation Loss: 0.0456, PSNR: 13.409769229062158
Epoch [6/20], Training Loss: 0.0347, Validation Loss: 0.0292, PSNR: 15.348392855845985
Epoch [7/20], Training Loss: 0.0278, Validation Loss: 0.0255, PSNR: 15.930064928716938
Epoch [8/20], Training Loss: 0.0251, Validation Loss: 0.0234, PSNR: 16.31067966883557
Epoch [9/20], Training Loss: 0.0288, Validation Loss: 0.0253, PSNR: 15.963336682591251
Epoch [10/20], Training Loss: 0.0240, Validation Loss: 0.0218, PSNR: 16.61931500950709
Epoch [11/20], Training Loss: 0.0219, Validat

[I 2023-06-22 15:49:22,735] Trial 10 finished with value: 0.017695380258373916 and parameters: {'lr': 0.009797605723301015, 'd': 36, 's': 20, 'm': 6}. Best is trial 6 with value: 0.01646598894149065.


Epoch [20/20], Training Loss: 0.0185, Validation Loss: 0.0177, PSNR: 17.52140100314453
Epoch [1/20], Training Loss: 0.3607, Validation Loss: 0.3344, PSNR: 4.75797848472722
Epoch [2/20], Training Loss: 0.3193, Validation Loss: 0.2520, PSNR: 5.9856126055378684
Epoch [3/20], Training Loss: 0.2006, Validation Loss: 0.1386, PSNR: 8.581184126634387
Epoch [4/20], Training Loss: 0.1107, Validation Loss: 0.0904, PSNR: 10.439453585297986
Epoch [5/20], Training Loss: 0.0884, Validation Loss: 0.0836, PSNR: 10.776255693145531
Epoch [6/20], Training Loss: 0.0838, Validation Loss: 0.0805, PSNR: 10.943209982051869
Epoch [7/20], Training Loss: 0.0808, Validation Loss: 0.0778, PSNR: 11.092910334642676
Epoch [8/20], Training Loss: 0.0781, Validation Loss: 0.0748, PSNR: 11.258514570787137
Epoch [9/20], Training Loss: 0.0750, Validation Loss: 0.0714, PSNR: 11.465073862533085
Epoch [10/20], Training Loss: 0.0711, Validation Loss: 0.0671, PSNR: 11.733410202958755
Epoch [11/20], Training Loss: 0.0664, Validat